In [7]:
import numpy as np
import pandas as pd
import seaborn as sns

import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import model_from_json

from keras.regularizers import l1

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import datetime

import plotly as py
import plotly.offline as pyo
from plotly.graph_objs import Data,Figure



In [2]:
# PREPROCESSING

# attach the column names to the dataset
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

# open the csv dataset
df = pd.read_csv("kddcup.data_10_percent.gz",compression='gzip',header=None, names = col_names)

# Remove smurf and neptune attacks
df = df[df.label != 'smurf.']
df = df[df.label != 'neptune.']

# Remove duplicates
df.drop_duplicates(subset=None, keep='first', inplace=True)


#data = df.drop("label", axis=1) #labels serviront pour évaluer la qualité du modèle
target = df["label"]

# Standardize/Normalize dataset?

# One hot encoding
cols_to_dummify = ['protocol_type','service','flag']
data_dummy = pd.get_dummies(df, columns=cols_to_dummify, prefix=cols_to_dummify)
target_dummy = pd.get_dummies(target)
target_categories = target_dummy.columns

# Splitting train/test
X = data_dummy.values
y = target_dummy.values
RANDOM_SEED = 101 #penser a changer la seed a posteriori pour voir
X_train, X_test = train_test_split(data_dummy, test_size=0.2, random_state = RANDOM_SEED)

X_train = X_train[X_train['label'] == 'normal.' ]
X_train = X_train.drop(['label'], axis=1)
y_test  = X_test['label']
X_test  = X_test.drop(['label'], axis=1)
X_train = X_train.values
X_test  = X_test.values
print('Training data size   :', X_train.shape)
print('Validation(test) data size :', X_test.shape)
df['label'].value_counts()
data_dummy.head()


Training data size   : (70272, 109)
Validation(test) data size : (18625, 109)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,181,5450,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [3]:
#MODEL BUILDING
input_dim = X_train.shape[1]
encoding_dim = 6

# Sequential
autoencoder_seq = Sequential()
autoencoder_seq.add(Dense(input_dim, input_shape=(input_dim,)))
autoencoder_seq.add(Dense(encoding_dim, activation="tanh"))
autoencoder_seq.add(Dense(int(encoding_dim/2),activation="tanh"))
autoencoder_seq.add(Dense(2,activation="tanh"))
autoencoder_seq.add(Dense(int(encoding_dim/2),activation="tanh"))
autoencoder_seq.add(Dense(encoding_dim,activation="tanh"))
autoencoder_seq.add(Dense(input_dim,activation="tanh"))
autoencoder_seq.compile(optimizer='adam', loss='binary_crossentropy',
             metrics=['accuracy'])
autoencoder_seq.summary()

# Functional 
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh",activity_regularizer=l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="tanh")(encoder)
encoder = Dense(int(2), activation="tanh")(encoder)
decoder = Dense(int(encoding_dim/ 2), activation='tanh')(encoder)
decoder = Dense(int(encoding_dim), activation='tanh')(decoder)
decoder = Dense(input_dim, activation='tanh')(decoder)
autoencoder_func = Model(inputs=input_layer, outputs=decoder)
autoencoder_func.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'] )
autoencoder_func.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 109)               11990     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 660       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 21        
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 9         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 24        
_________________________________________________________________
dens

In [9]:
# MODEL TRAINING

nb_epoch = 20
batch_size = 50


t_ini = datetime.datetime.now()
history = autoencoder_func.fit(X_train, X_train,
                        validation_data= (X_test,X_test),
                        epochs=nb_epoch,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_split=0.1
                        )

t_fin = datetime.datetime.now()
print('Time to run the model: {} Sec.'.format((t_fin - t_ini).total_seconds()))


df_history = pd.DataFrame(history.history)

Train on 70272 samples, validate on 18625 samples
Epoch 1/20
70272/70272 [==============================] - 2s 35us/step - loss: -152.8523 - acc: 0.8843 - val_loss: -663.5535 - val_acc: 0.8947
Epoch 2/20
70272/70272 [==============================] - 2s 32us/step - loss: -637.4503 - acc: 0.8962 - val_loss: -1143.7218 - val_acc: 0.8947
Epoch 3/20
70272/70272 [==============================] - 2s 31us/step - loss: -754.9940 - acc: 0.8962 - val_loss: -1426.7173 - val_acc: 0.8947
Epoch 4/20
70272/70272 [==============================] - 2s 34us/step - loss: -808.5053 - acc: 0.8962 - val_loss: -1558.5271 - val_acc: 0.8947
Epoch 5/20
70272/70272 [==============================] - 2s 33us/step - loss: -815.2979 - acc: 0.8962 - val_loss: -1558.5335 - val_acc: 0.8947
Epoch 6/20
70272/70272 [==============================] - 2s 33us/step - loss: -815.7092 - acc: 0.8962 - val_loss: -1558.5178 - val_acc: 0.8947
Epoch 7/20
70272/70272 [==============================] - 2s 34us/step - loss: -815.705

In [20]:
# MODEL EVALUATION

pyo.init_notebook_mode(connected=True)
trace = []
for label, loss in zip(['Train', 'Validation'], ['loss', 'val_loss']):
    trace0 = {'type' : 'scatter', 
              'x'    : df_history.index.tolist(),
              'y'    : df_history[loss].tolist(),
              'name' : label,
              'mode' : 'lines'
              }
        
    trace.append(trace0)
data = Data(trace)
layout = {'title' : 'Model train-vs-validation loss', 'titlefont':{'size' : 30},
          'xaxis' : {'title':  '<b> Epochs', 'titlefont':{ 'size' : 25}},
          'yaxis' : {'title':  '<b> Loss', 'titlefont':{ 'size' : 25}},
          }
fig = Figure(data = data, layout = layout)
    
pyo.iplot(fig)